In [750]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [751]:
data_path = r'C:\Users\91790\Desktop\ML\Project\Machine-Learning--Projects\Projects\Projects for Submission\Project 1 - Mercedes-Benz Greener Manufacturing\Dataset for the project\train\train.csv'
data = pd.read_csv(data_path)

test_data_path = r'C:\Users\91790\Desktop\ML\Project\Machine-Learning--Projects\Projects\Projects for Submission\Project 1 - Mercedes-Benz Greener Manufacturing\Dataset for the project\test\test.csv'
unseen_data = pd.read_csv(test_data_path)

In [752]:
#Data Structure
print('Shape of {name} : {shape}'.format(name = "Training data",shape = data.shape))
print('Null values column wise: ', data.columns[data.isnull().sum() > 0])
print('All unique datatypes: {d_types}'.format(d_types = data.dtypes.unique()))
data_copy = data.copy()

Shape of Training data : (4209, 378)
Null values column wise:  Index([], dtype='object')
All unique datatypes: [dtype('int64') dtype('float64') dtype('O')]


In [753]:
#Get columns with object datatype
objtype_col = data.columns[data.dtypes == object]
objtype_col

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [754]:
# Testing purpose - label encode object type data
def encode_array(arr):
    l_encoder = preprocessing.LabelEncoder().fit(arr)
    return l_encoder.transform(arr)

In [755]:
# Label Encode Training Copied Data
for col in objtype_col:
    data_copy[col] = encode_array(data_copy[col])
print(data_copy.head())

# Label Encode Unseen Test data
for col in objtype_col:
    unseen_data[col] = encode_array(unseen_data[col])
print(unseen_data.head())

   ID       y  X0  X1  X2  X3  X4  X5  X6  X8  ...  X375  X376  X377  X378  \
0   0  130.81  32  23  17   0   3  24   9  14  ...     0     0     1     0   
1   6   88.53  32  21  19   4   3  28  11  14  ...     1     0     0     0   
2   7   76.26  20  24  34   2   3  27   9  23  ...     0     0     0     0   
3   9   80.62  20  21  34   5   3  27  11   4  ...     0     0     0     0   
4  13   78.02  20  23  34   5   3  12   3  13  ...     0     0     0     0   

   X379  X380  X382  X383  X384  X385  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     1     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 378 columns]
   ID  X0  X1  X2  X3  X4  X5  X6  X8  X10  ...  X375  X376  X377  X378  X379  \
0   1  21  23  34   5   3  26   0  22    0  ...     0     0     0     1     0   
1   2  42   3   8   0   3   9   6  24    0  ...     0     0     1     0     0   
2   3  21  23  17   5   

In [756]:
#  Implementing Label encode on object type data
def encode_array(arr):
    l_encoder = preprocessing.LabelEncoder().fit(arr)
    return l_encoder.transform(arr)

for col in objtype_col:
    data[col] = encode_array(data[col])
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,32,23,17,0,3,24,9,14,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,32,21,19,4,3,28,11,14,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,20,24,34,2,3,27,9,23,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,20,21,34,5,3,27,11,4,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,20,23,34,5,3,12,3,13,...,0,0,0,0,0,0,0,0,0,0


In [757]:
#Columns having 0 variance/ const value
zero_var_col = data.columns[data.var(axis = 0) == 0]
zero_var_col

Index(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293',
       'X297', 'X330', 'X347'],
      dtype='object')

In [758]:
#Drop col with zero variance on data and data_copy
data_copy.drop(columns=zero_var_col, inplace=True)
data.drop(columns=zero_var_col, inplace=True)
print("Train Data:",data.shape)

# Dropping same row in unseen data
unseen_data.drop(columns=zero_var_col, inplace=True)
print("Unseen Test Data:",unseen_data.shape)

Train Data: (4209, 366)
Unseen Test Data: (4209, 365)


In [759]:
#Separating features and target variable
train_x, train_y = data.loc[:,'X0':], data.loc[:,'y'].values
print("Features: ", train_x.head())
print("Target Variable: ", train_y[:5])

# Removing ID column from unseen data
unseen_data = unseen_data.loc[:,'X0':]

Features:     X0  X1  X2  X3  X4  X5  X6  X8  X10  X12  ...  X375  X376  X377  X378  \
0  32  23  17   0   3  24   9  14    0    0  ...     0     0     1     0   
1  32  21  19   4   3  28  11  14    0    0  ...     1     0     0     0   
2  20  24  34   2   3  27   9  23    0    0  ...     0     0     0     0   
3  20  21  34   5   3  27  11   4    0    0  ...     0     0     0     0   
4  20  23  34   5   3  12   3  13    0    0  ...     0     0     0     0   

   X379  X380  X382  X383  X384  X385  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     1     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 364 columns]
Target Variable:  [130.81  88.53  76.26  80.62  78.02]


In [760]:
# Splitting data into training and testing set
train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(train_x,train_y,test_size = 0.20)
print("Training features set:", train_x_split.shape)
print("Testing features set:", test_x_split.shape)
train_x_split.head()

Training features set: (3367, 364)
Testing features set: (842, 364)


,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
3473,8,20,16,6,3,22,6,15,0,0,...,1,0,0,0,0,0,0,0,0,0
2827,19,1,16,0,3,20,2,9,0,0,...,0,0,1,0,0,0,0,0,0,0
1291,40,3,33,2,3,2,9,17,0,0,...,0,0,1,0,0,0,0,0,0,0
2397,44,1,16,2,3,18,9,21,0,0,...,1,0,0,0,0,0,0,0,0,0
503,46,1,7,6,3,9,8,9,0,0,...,1,0,0,0,0,0,0,0,0,0


In [761]:
# Standardizing Train Data before applying PCA
cols = train_x_split.columns
std = StandardScaler()
std.fit(train_x_split)
train_x_split = pd.DataFrame(std.transform(train_x_split), columns=cols)
print("After standardizing:")
print(train_x_split.head())

After standardizing:
         X0        X1        X2        X3        X4        X5        X6  \
0 -1.576692  1.049492 -0.127170  1.769526  0.027646  1.049848 -0.290191   
1 -0.777798 -1.184458 -0.127170 -1.679052  0.027646  0.806787 -1.664897   
2  0.747362 -0.949306  1.430852 -0.529526  0.027646 -1.380763  0.740839   
3  1.037869 -1.184458 -0.127170 -0.529526  0.027646  0.563726  0.740839   
4  1.183123 -1.184458 -0.952005  1.769526  0.027646 -0.530049  0.397162   

         X8       X10       X12  ...      X375     X376      X377     X378  \
0  0.479187 -0.117691 -0.286253  ...  1.467182 -0.24181 -0.686257 -0.14571   
1 -0.374626 -0.117691 -0.286253  ... -0.681579 -0.24181  1.457181 -0.14571   
2  0.763791 -0.117691 -0.286253  ... -0.681579 -0.24181  1.457181 -0.14571   
3  1.332999 -0.117691 -0.286253  ...  1.467182 -0.24181 -0.686257 -0.14571   
4 -0.374626 -0.117691 -0.286253  ...  1.467182 -0.24181 -0.686257 -0.14571   

      X379      X380      X382      X383      X384      X38

In [762]:
'''# Standardizing test data before applying PCA
cols = test_x_split.columns
std = StandardScaler()
std.fit(test_x_split)
test_x_split = pd.DataFrame(std.transform(test_x_split), columns=cols)
print("After standardizing:")
print(test_x_split.head())'''

'# Standardizing test data before applying PCA\ncols = test_x_split.columns\nstd = StandardScaler()\nstd.fit(test_x_split)\ntest_x_split = pd.DataFrame(std.transform(test_x_split), columns=cols)\nprint("After standardizing:")\nprint(test_x_split.head())'

In [763]:
# Standardizing Unseen test data before applying PCA
unseen_cols = unseen_data.columns
unseen_std = StandardScaler()
unseen_std.fit(unseen_data)
unseen_data = pd.DataFrame(unseen_std.transform(unseen_data), columns=unseen_cols)
print("After standardizing:")
print(unseen_data.head())

After standardizing:
         X0        X1        X2        X3        X4        X5        X6  \
0 -0.625211  1.395760  1.586068  1.163082  0.036299  1.266652 -2.388888   
1  0.754609 -0.945199 -0.956445 -1.651020  0.036299 -0.695011 -0.296602   
2 -0.625211  1.395760 -0.076345  1.163082  0.036299 -1.733538  0.749541   
3 -0.625211  0.225281  1.586068  1.163082  0.036299  1.843611  1.446970   
4  0.951726  1.044616 -0.076345 -0.525379  0.036299  1.728219  0.400827   

         X8       X10       X12  ...      X375      X376      X377      X378  \
0  1.488396 -0.139195 -0.283441  ... -0.695420 -0.228583 -0.673338  7.138835   
1  1.773477 -0.139195 -0.283441  ... -0.695420 -0.228583  1.485138 -0.140079   
2 -0.364632 -0.139195 -0.283441  ... -0.695420 -0.228583 -0.673338  7.138835   
3  0.205531 -0.139195 -0.283441  ... -0.695420 -0.228583 -0.673338  7.138835   
4  0.062990 -0.139195 -0.283441  ...  1.437979 -0.228583 -0.673338 -0.140079   

       X379      X380      X382      X383      

In [764]:
#Applying standardization using training set inforamtion
test_x_split = pd.DataFrame(std.transform(test_x_split), columns=cols)
print("After standardizing:")
print(test_x_split.head())

After standardizing:
         X0        X1        X2        X3        X4        X5        X6  \
0  1.037869 -1.184458 -0.127170 -0.529526  0.027646 -0.651580  0.740839   
1  0.384229  1.049492 -0.860357  1.769526  0.027646  1.535970 -0.290191   
2  1.183123  1.754950 -0.952005 -0.529526  0.027646  1.414439 -1.321221   
3  0.456856  1.402221 -0.127170  0.045237  0.027646  1.414439 -0.633867   
4 -1.576692  1.049492 -0.127170  0.045237  0.027646 -0.894641 -0.290191   

         X8       X10       X12  ...      X375      X376      X377     X378  \
0  0.906093 -0.117691 -0.286253  ... -0.681579  4.135475 -0.686257 -0.14571   
1  0.336884 -0.117691  3.493411  ...  1.467182 -0.241810 -0.686257 -0.14571   
2 -0.374626 -0.117691 -0.286253  ...  1.467182 -0.241810 -0.686257 -0.14571   
3 -0.516928 -0.117691 -0.286253  ... -0.681579 -0.241810 -0.686257 -0.14571   
4 -0.516928 -0.117691 -0.286253  ...  1.467182 -0.241810 -0.686257 -0.14571   

      X379      X380      X382      X383      X384   

In [765]:
# PCA

'''pca = decomposition.PCA(n_components=150)
pca.fit(train_x_split)

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of componenets")
plt.ylabel("Variance")
plt.title("Variance Growth")
plt.show()

#By seeing above trend, even if we choose 150 number of componenets, it will cover 99% of variance
pca = decomposition.PCA(n_components=150)
pca.fit(train_x_split)

pca_labels = ['pca_'+str(i) for i in range(1,151)]
pca_train_x_split = pd.DataFrame(data=pca.transform(train_x_split), columns=pca_labels)
print("Transformed train set shape: ",pca_train_x_split.shape)

#PCA on test set
pca_test = decomposition.PCA(n_components=150)
pca.fit(test_x_split)

pca_test_x_split = pd.DataFrame(data=pca.transform(test_x_split),columns=pca_labels)
print("Transformed test set shape: ",pca_test_x_split.shape)'''

'pca = decomposition.PCA(n_components=150)\npca.fit(train_x_split)\n\nplt.figure()\nplt.plot(np.cumsum(pca.explained_variance_ratio_))\nplt.xlabel("Number of componenets")\nplt.ylabel("Variance")\nplt.title("Variance Growth")\nplt.show()\n\n#By seeing above trend, even if we choose 150 number of componenets, it will cover 99% of variance\npca = decomposition.PCA(n_components=150)\npca.fit(train_x_split)\n\npca_labels = [\'pca_\'+str(i) for i in range(1,151)]\npca_train_x_split = pd.DataFrame(data=pca.transform(train_x_split), columns=pca_labels)\nprint("Transformed train set shape: ",pca_train_x_split.shape)\n\n#PCA on test set\npca_test = decomposition.PCA(n_components=150)\npca.fit(test_x_split)\n\npca_test_x_split = pd.DataFrame(data=pca.transform(test_x_split),columns=pca_labels)\nprint("Transformed test set shape: ",pca_test_x_split.shape)'

In [766]:
'''# PCA on unseen data

unseen_pca = decomposition.PCA(n_components=150)
pca.fit(unseen_data)

#unseen_pca_labels = ['pca_'+str(i) for i in range(1,151)]
pca_unseen_data = pd.DataFrame(data=pca.transform(unseen_data), columns=pca_labels)
print("Transformed unseen data shape: ",pca_unseen_data.shape)'''

'# PCA on unseen data\n\nunseen_pca = decomposition.PCA(n_components=150)\npca.fit(unseen_data)\n\n#unseen_pca_labels = [\'pca_\'+str(i) for i in range(1,151)]\npca_unseen_data = pd.DataFrame(data=pca.transform(unseen_data), columns=pca_labels)\nprint("Transformed unseen data shape: ",pca_unseen_data.shape)'

In [767]:
xgb_train_set = xgb.DMatrix(train_x_split, label=train_y_split)
xgb_test_set = xgb.DMatrix(test_x_split, label=test_y_split)

In [768]:
params = {}

params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2score(preds, d_train):
    labels = d_train.get_label()
    return 'r2',r2_score(labels,preds)

watchlist = [(xgb_train_set,'train'),(xgb_test_set,'test')]

clf = xgb.train(params,xgb_train_set,1000,watchlist,early_stopping_rounds=50,feval=xgb_r2score,maximize=True,verbose_eval=10)

[12:02:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:98.99657	test-rmse:98.83813	train-r2:-59.45368	test-r2:-61.95206
Multiple eval metrics have been passed: 'test-r2' will be used for early stopping.

Will train until test-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.06319	test-rmse:80.84130	train-r2:-39.53495	test-r2:-41.11409
[20]	train-rmse:66.43913	test-rmse:66.15540	train-r2:-26.22890	test-r2:-27.20275
[30]	train-rmse:54.52700	test-rmse:54.18199	train-r2:-17.34026	test-r2:-17.91781
[40]	train-rmse:44.83980	test-rmse:44.43317	train-r2:-11.40251	test-r2:-11.72259
[50]	train-rmse:36.98085	test-rmse:36.51025	train-r2:-7.43599	test-r2:-7.58995
[60]	train-rmse:30.62636	test-rmse:30.09397	train-r2:-4.78593	test-r2:-4.83606
[70]	train-rmse:25.51157	test-rmse:24.91364	train-r2:-3.01473	test-r2:-2.99977
[80]	train-rmse:21.42103	test-rmse:20.75277

In [770]:
# Predictions
dmat_test_set = xgb.DMatrix(unseen_data)
pred_test_set = clf.predict(dmat_test_set)

ids = [i for i in range(1,4210)]
result_pred = pd.DataFrame(data = ids, columns=['ID'])
result_pred['y'] = pred_test_set

In [684]:
result_pred.to_csv(r'C:\Users\91790\Desktop\ML\Project\Machine-Learning--Projects\Projects\Projects for Submission\Project 1 - Mercedes-Benz Greener Manufacturing\Dataset for the project\result.csv',index = False)

In [677]:
result_pred.head()

,ID,y
0,1,84.878258
1,2,104.587158
2,3,84.465904
3,4,90.860306
4,5,95.146538
